# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

In [3]:
findspark.init('/home/quanttraderkim/spark-2.3.1-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [8]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [9]:
df.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [10]:
df.columns # Age, totalpur, years, num_sites

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [11]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [17]:
process_data  = df.select('Age','Total_Purchase', 'Years','Num_Sites', 'Churn')

In [18]:
process_data.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



## model

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [16]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer

In [20]:
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase', 'Years','Num_Sites'], outputCol='features')

In [15]:
model = LogisticRegression(labelCol='Churn')

In [25]:
pipeline = Pipeline(stages = [assembler, model])

In [26]:
train_data, test_data = process_data.randomSplit([0.7,0.3])

In [27]:
fitted_model = pipeline.fit(train_data)

In [28]:
pred = fitted_model.transform(test_data)

In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [31]:
pred.columns

['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Churn',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [33]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol= 'Churn')

In [55]:
evaluator.evaluate(pred)

0.7464607464607464

## Prediction on new data

In this stage, we are going to deploy our model. So we gonna fit all historic data we have, and then predict labels for new data. 

In [36]:
new_data = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [57]:
final_fitted = pipeline.fit(process_data)

In [58]:
prediction = final_fitted.transform(new_data)

In [60]:
#prediction.describe().show()

In [64]:
#prediction.show()

In [61]:
prediction_results = prediction.select('Names','prediction','Company').collect()

In [62]:
prediction_results

[Row(Names='Andrew Mccall', prediction=0.0, Company='King Ltd'),
 Row(Names='Michele Wright', prediction=1.0, Company='Cannon-Benson'),
 Row(Names='Jeremy Chang', prediction=1.0, Company='Barron-Robertson'),
 Row(Names='Megan Ferguson', prediction=1.0, Company='Sexton-Golden'),
 Row(Names='Taylor Young', prediction=0.0, Company='Wood LLC'),
 Row(Names='Jessica Drake', prediction=1.0, Company='Parks-Robbins')]